# Zero-Cost Proxy Evaluation  w/ minibatch

Testing zero-cost proxies with zone-agnostic behavior policy model

In [1]:
import numpy as np
import sys
import matplotlib.pyplot as plt
import pandas as pd
import pickle
sys.path.insert(0, "../")
import torch
from ppo import PPO
import glob
from tqdm.notebook import tqdm
import json
from scipy.stats import spearmanr

In [2]:
# log_data = pd.read_csv("../data/rule_based_log_data/doee/0_cleaned_log.csv")
# with open("../data/rule_based_log_data/doee/action_probs_all_data.pkl", "rb") as f:
#     behavior_model = pickle.load(f)
# # with open("../data/invalid_policy_list_20220705.json") as f:
# #     invalid_policies = json.load(f)["invalid_policies"]
# # invalid_policies = []

log_data = pd.read_csv("../data/rule_based_log_data/15zone/0_cleaned_log.csv")
with open("../data/rule_based_log_data/15zone/action_probs_all_data.pkl", "rb") as f:
    behavior_model = pickle.load(f)
# with open("../data/invalid_policy_list_20220705.json") as f:
#     invalid_policies = json.load(f)["invalid_policies"]
invalid_policies = []

## 30 Day MiniBatch

In [3]:
num_ts_per_day = 4 * 24
num_days = 15
ts_end = num_ts_per_day * num_days
zones = log_data["zone"].unique()

## Loading all Policies

In [4]:
policy_list = sorted(list(glob.glob(f"../policy_library_20220820/**.pth")))

### 1. GradNorm

In [5]:
from zero_cost_proxies.grad_norm import GradNorm
policy_scores = {}
for zone in zones:
    print(zone)
    ope_data = log_data[log_data["zone"] == zone].sort_values(by=["timestep"])
    ope_data = log_data[:ts_end]
    for policy in tqdm(policy_list):
        agent = PPO(6, 1, 0.003, 0.0005, 1, 10, 0.2,
                    has_continuous_action_space=True, action_std_init=0.2, 
                    device=torch.device('cpu'), diverse_policies=list(),
                    diverse_weight=0, diverse_increase=True)
        agent.load(policy)
        agent.policy_evaluation = True
        agent.policy_old.set_action_std(0.1)
        gn = GradNorm(agent, behavior_model)
        if policy not in invalid_policies:
            norm = gn.get_grad_norm(ope_data)
        else:
            continue
        if policy not in policy_scores:
            policy_scores[policy] = {}
        if zone not in policy_scores[policy]:
            policy_scores[policy][zone] = norm

Perimeter_top_ZN_3


  0%|          | 0/870 [00:00<?, ?it/s]

/Users/aakashsasikumar/Documents/MSc/Research/Dissimilar Policy/building-MARL/studies/../ope/iw.py:61: RuntimeWarning: divide by zero encountered in log
  return np.log(value / (1- value))


Perimeter_bot_ZN_1


  0%|          | 0/870 [00:00<?, ?it/s]

...

  0%|          | 0/870 [00:00<?, ?it/s]

Perimeter_mid_ZN_1


  0%|          | 0/870 [00:00<?, ?it/s]

Perimeter_bot_ZN_4


  0%|          | 0/870 [00:00<?, ?it/s]

In [6]:
eval_data_loc = "../data/evaluation_report_20220820.csv"
df = pd.read_csv(eval_data_loc, header=None, names=["datetime","policy","zone","energy"])
spearman_corr = {}
for zone in zones:
    eval_df = df[df["zone"] == zone]

    # invalid_policies = list(set(policy_list) - set(policy_scores.keys()))
    for i_policy in invalid_policies:
        eval_df = eval_df[eval_df["policy"]!=i_policy[3:]]
    eval_df = eval_df.sort_values(by=["energy"])
    score_list = []
    for i, row in eval_df.iterrows():
        score_list.append(policy_scores[f"../{row['policy']}"][zone])

    eval_df["ope_scores"] = score_list
    correlation = spearmanr(eval_df["energy"].values, eval_df["ope_scores"].values)
    spearman_corr[zone] = correlation

In [7]:
spearman_corr

{'Perimeter_top_ZN_3': SpearmanrResult(correlation=-0.6865461379890686, pvalue=2.830810891363366e-122),
 'Perimeter_bot_ZN_1': SpearmanrResult(correlation=-0.6369612189430071, pvalue=3.4672597559123844e-100),
 'Perimeter_top_ZN_1': SpearmanrResult(correlation=-0.6257318273268654, pvalue=9.817431228826295e-96),
 'Core_bottom': SpearmanrResult(correlation=-0.6209011775719514, pvalue=7.106159124851904e-94),
 'Perimeter_top_ZN_4': SpearmanrResult(correlation=-0.6330619441564675, pvalue=1.2787197136151444e-98),
 'Core_top': SpearmanrResult(correlation=-0.6300092341279773, pvalue=2.0786685537682936e-97),
 'Perimeter_bot_ZN_2': SpearmanrResult(correlation=-0.6305946149171981, pvalue=1.2207197934204085e-97),
 'Perimeter_bot_ZN_3': SpearmanrResult(correlation=-0.7265522434160954, pvalue=1.2891916550609763e-143),
 'Perimeter_mid_ZN_4': SpearmanrResult(correlation=-0.6322384578781809, pvalue=2.7213989812298362e-98),
 'Core_mid': SpearmanrResult(correlation=-0.6222749693766, pvalue=2.1187868081180

In [11]:
with open(f"data/15zone/grad_norm/raw_scores/gradnorm_raw_scores_{num_days}_days_06_09_2022.pkl", "wb+") as f:
    pickle.dump(policy_scores, f)

In [12]:
with open(f"data/15zone/grad_norm/spearman_corr/gradnorm_spearman_corr_{num_days}_days_06_09_2022.pkl", "wb+") as f:
    pickle.dump(spearman_corr, f)

### 2. Synflow

In [9]:
from zero_cost_proxies.synflow import SynFlow
policy_scores_synflow = {}
for zone in zones:
    print(zone)
    ope_data = log_data[log_data["zone"] == zone].sort_values(by=["timestep"])
    ope_data = log_data[:ts_end]
    for policy in tqdm(policy_list):
        agent = PPO(6, 1, 0.003, 0.0005, 1, 10, 0.2,
                    has_continuous_action_space=True, action_std_init=0.2, 
                    device=torch.device('cpu'), diverse_policies=list(),
                    diverse_weight=0, diverse_increase=True)
        agent.load(policy)
        agent.policy_evaluation = True
        agent.policy_old.set_action_std(0.1)
        sf = SynFlow(agent)
        if policy not in invalid_policies:
            synflow = sf.get_synflow()
        else:
            continue
        if policy not in policy_scores_synflow:
            policy_scores_synflow[policy] = {}
        if zone not in policy_scores_synflow[policy]:
            policy_scores_synflow[policy][zone] = synflow

Perimeter_top_ZN_3


  0%|          | 0/800 [00:00<?, ?it/s]

Perimeter_bot_ZN_1


  0%|          | 0/800 [00:00<?, ?it/s]

Perimeter_top_ZN_1


...

  0%|          | 0/800 [00:00<?, ?it/s]

Perimeter_mid_ZN_1


  0%|          | 0/800 [00:00<?, ?it/s]

Perimeter_bot_ZN_4


  0%|          | 0/800 [00:00<?, ?it/s]

In [10]:
eval_data_loc = "../data/1month_eval.csv"
df = pd.read_csv(eval_data_loc, header=None, names=["datetime","policy","zone","energy"])
spearman_corr_synflow = {}
for zone in zones:
    eval_df = df[df["zone"] == zone]

    # invalid_policies = list(set(policy_list) - set(policy_scores.keys()))
    for i_policy in invalid_policies:
        eval_df = eval_df[eval_df["policy"]!=i_policy[3:]]
    eval_df = eval_df.sort_values(by=["energy"])
    score_list = []
    for i, row in eval_df.iterrows():
        score_list.append(policy_scores_synflow[f"../{row['policy']}"][zone])

    eval_df["ope_scores"] = score_list
    correlation = spearmanr(eval_df["energy"].values, eval_df["ope_scores"].values)
    spearman_corr_synflow[zone] = correlation

In [11]:
spearman_corr_synflow

{'Perimeter_top_ZN_3': SpearmanrResult(correlation=-0.02399655788801756, pvalue=0.5709350401752371),
 'Perimeter_bot_ZN_1': SpearmanrResult(correlation=-0.007958102162903363, pvalue=0.8510831000493045),
 'Perimeter_top_ZN_1': SpearmanrResult(correlation=0.004610768076634543, pvalue=0.9133850920486921),
 'Core_bottom': SpearmanrResult(correlation=0.0846908842595264, pvalue=0.04534104009312232),
 'Perimeter_top_ZN_4': SpearmanrResult(correlation=0.01057283551657145, pvalue=0.8030336897038484),
 'Core_top': SpearmanrResult(correlation=0.056829866807361164, pvalue=0.17968820761249374),
 'Perimeter_bot_ZN_2': SpearmanrResult(correlation=-0.005958563816778369, pvalue=0.8882133629244287),
 'Perimeter_bot_ZN_3': SpearmanrResult(correlation=0.0023340313100253082, pvalue=0.9560902859952176),
 'Perimeter_mid_ZN_4': SpearmanrResult(correlation=-0.03969206587361125, pvalue=0.34890507616128863),
 'Core_mid': SpearmanrResult(correlation=0.06361473610160781, pvalue=0.13304293548202217),
 'Perimeter_mi

In [12]:
with open("data/synflow_spearman_corr15_06_2022.pkl", "wb+") as f:
    pickle.dump(spearman_corr_synflow, f)

### 3. Jacobian Covariance

In [5]:
from zero_cost_proxies.jacob_cov import JacobianCovariance
policy_scores_jc = {}
for zone in zones:
    print(zone)
    ope_data = log_data[log_data["zone"] == zone].sort_values(by=["timestep"])
    ope_data = log_data[:ts_end]
    states = []
    for i, row in ope_data.iterrows():
        state_vars = ["outdoor_temp", "solar_irradiation", "time_hour",
                      "zone_humidity", "zone_temp", "zone_occupancy"]
        state = [row[var] for var in state_vars]
        states.append(state)
    states = torch.Tensor(states)
    for policy in tqdm(policy_list):
        agent = PPO(6, 1, 0.003, 0.0005, 1, 10, 0.2,
                    has_continuous_action_space=True, action_std_init=0.2, 
                    device=torch.device('cpu'), diverse_policies=list(),
                    diverse_weight=0, diverse_increase=True)
        agent.load(policy)
        agent.policy_evaluation = True
        agent.policy_old.set_action_std(0.1)
        jc = JacobianCovariance(agent)
        if policy not in invalid_policies:
            synflow = jc.compute_jacob_cov(states)
        else:
            continue
        if policy not in policy_scores_jc:
            policy_scores_jc[policy] = {}
        if zone not in policy_scores_jc[policy]:
            policy_scores_jc[policy][zone] = synflow

Perimeter_top_ZN_3


  0%|          | 0/800 [00:00<?, ?it/s]

Perimeter_bot_ZN_1


  0%|          | 0/800 [00:00<?, ?it/s]

Perimeter_top_ZN_1


...

  0%|          | 0/800 [00:00<?, ?it/s]

Perimeter_mid_ZN_1


  0%|          | 0/800 [00:00<?, ?it/s]

Perimeter_bot_ZN_4


  0%|          | 0/800 [00:00<?, ?it/s]

In [6]:
eval_data_loc = "../data/1month_eval.csv"
df = pd.read_csv(eval_data_loc, header=None, names=["datetime","policy","zone","energy"])
spearman_corr_jc = {}
for zone in zones:
    eval_df = df[df["zone"] == zone]

    # invalid_policies = list(set(policy_list) - set(policy_scores.keys()))
    for i_policy in invalid_policies:
        eval_df = eval_df[eval_df["policy"]!=i_policy[3:]]
    eval_df = eval_df.sort_values(by=["energy"])
    score_list = []
    for i, row in eval_df.iterrows():
        score_list.append(policy_scores_jc[f"../{row['policy']}"][zone])

    eval_df["ope_scores"] = score_list
    correlation = spearmanr(eval_df["energy"].values, eval_df["ope_scores"].values)
    spearman_corr_jc[zone] = correlation

In [7]:
spearman_corr_jc

{'Perimeter_top_ZN_3': SpearmanrResult(correlation=-0.010449571127394685, pvalue=0.8051123996789393),
 'Perimeter_bot_ZN_1': SpearmanrResult(correlation=0.04045049723969454, pvalue=0.33976566455830204),
 'Perimeter_top_ZN_1': SpearmanrResult(correlation=0.0025946733936231306, pvalue=0.9511926704797761),
 'Core_bottom': SpearmanrResult(correlation=0.005884850333462486, pvalue=0.8895873841026646),
 'Perimeter_top_ZN_4': SpearmanrResult(correlation=0.0204017350491469, pvalue=0.6302803636181593),
 'Core_top': SpearmanrResult(correlation=0.005199273808824353, pvalue=0.9023818561397574),
 'Perimeter_bot_ZN_2': SpearmanrResult(correlation=0.0006065389973867092, pvalue=0.9885839284050955),
 'Perimeter_bot_ZN_3': SpearmanrResult(correlation=-0.017793115861382925, pvalue=0.6746480699322992),
 'Perimeter_mid_ZN_4': SpearmanrResult(correlation=-0.018052040792685135, pvalue=0.6701901979459284),
 'Core_mid': SpearmanrResult(correlation=-0.015886595275192424, pvalue=0.7078147788801797),
 'Perimeter_m

## 4. SNIP

In [5]:
from zero_cost_proxies.snip import SNIP
policy_scores_snip = {}
for zone in zones:
    print(zone)
    ope_data = log_data[log_data["zone"] == zone].sort_values(by=["timestep"])
    ope_data = log_data[:ts_end]
    for policy in tqdm(policy_list):
        agent = PPO(6, 1, 0.003, 0.0005, 1, 10, 0.2,
                    has_continuous_action_space=True, action_std_init=0.2, 
                    device=torch.device('cpu'), diverse_policies=list(),
                    diverse_weight=0, diverse_increase=True)
        agent.load(policy)
        agent.policy_evaluation = True
        agent.policy_old.set_action_std(0.1)
        snip = SNIP(agent, behavior_model)
        if policy not in invalid_policies:
            sn = snip.compute_snip(ope_data)
        else:
            continue
        if policy not in policy_scores_snip:
            policy_scores_snip[policy] = {}
        if zone not in policy_scores_snip[policy]:
            policy_scores_snip[policy][zone] = sn

Perimeter_top_ZN_3


  0%|          | 0/870 [00:00<?, ?it/s]

/Users/aakashsasikumar/Documents/MSc/Research/Dissimilar Policy/building-MARL/studies/../ope/iw.py:61: RuntimeWarning: divide by zero encountered in log
  return np.log(value / (1- value))


Perimeter_bot_ZN_1


  0%|          | 0/870 [00:00<?, ?it/s]

...

  0%|          | 0/870 [00:00<?, ?it/s]

Perimeter_mid_ZN_1


  0%|          | 0/870 [00:00<?, ?it/s]

Perimeter_bot_ZN_4


  0%|          | 0/870 [00:00<?, ?it/s]

In [6]:
eval_data_loc = "../data/eval_data/evaluation_report_20220820.csv"
df = pd.read_csv(eval_data_loc, header=None, names=["datetime","policy","zone","energy"])
spearman_corr_snip = {}
for zone in zones:
    eval_df = df[df["zone"] == zone]

    # invalid_policies = list(set(policy_list) - set(policy_scores.keys()))
    for i_policy in invalid_policies:
        eval_df = eval_df[eval_df["policy"]!=i_policy[3:]]
    eval_df = eval_df.sort_values(by=["energy"])
    score_list = []
    for i, row in eval_df.iterrows():
        score_list.append(policy_scores_snip[f"../{row['policy']}"][zone])

    eval_df["ope_scores"] = score_list
    correlation = spearmanr(eval_df["energy"].values, eval_df["ope_scores"].values)
    spearman_corr_snip[zone] = correlation

In [8]:
with open(f"data/15zone/snip/spearman_corr/snip_spearman_corr_{num_days}_days_18_09_2022.pkl", "wb+") as f:
    pickle.dump(spearman_corr_snip, f)

In [9]:
with open(f"data/15zone/snip/raw_scores/snip_raw_scores_{num_days}_days_18_09_2022.pkl", "wb+") as f:
    pickle.dump(policy_scores_snip, f)

In [7]:
spearman_corr_snip

{'Perimeter_top_ZN_3': SpearmanrResult(correlation=-0.703609201947425, pvalue=6.145721202954794e-131),
 'Perimeter_bot_ZN_1': SpearmanrResult(correlation=-0.6542677284841102, pvalue=2.014542520570895e-107),
 'Perimeter_top_ZN_1': SpearmanrResult(correlation=-0.6429933630539366, pvalue=1.1778615675130288e-102),
 'Core_bottom': SpearmanrResult(correlation=-0.6380135679971297, pvalue=1.2979226034588598e-100),
 'Perimeter_top_ZN_4': SpearmanrResult(correlation=-0.6492125052286722, pvalue=2.9287281926295602e-105),
 'Core_top': SpearmanrResult(correlation=-0.6465556174660639, pvalue=3.861597881400266e-104),
 'Perimeter_bot_ZN_2': SpearmanrResult(correlation=-0.6470613817535088, pvalue=2.3681563094549035e-104),
 'Perimeter_bot_ZN_3': SpearmanrResult(correlation=-0.738009865860827, pvalue=1.9160657493383974e-150),
 'Perimeter_mid_ZN_4': SpearmanrResult(correlation=-0.6484877425942756, pvalue=5.933859276103886e-105),
 'Core_mid': SpearmanrResult(correlation=-0.6391503963141589, pvalue=4.4707228